In [41]:
import pandas as pd
import numpy as np
import networkx as nx

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import tensorflow as tf

In [42]:
# test area
#x = np.array([[1, 1, 3], [2, 2, 3]])
#x = x[:, [i for i in range(x.shape[1]) if i not in [0, 1]]]

In [245]:
import pandas as pd
import numpy as np
def readF(file_path, delimiter='\t', header=None, y_index=-1, ignore_columns=None):
    '''
    @ Params
        file_path: a string indicate the path of the file
        delimiter: the delimiter for delimite the file, default is '\t'
        header: default is none
        y_index: the index of labels, default is the last one
        ignore_columns: a list of indexs representing correponding columns that need to be ignored

    @ Output
        X and y in terms of numpy array
    '''
    features = pd.read_csv(file_path, header=header, delimiter=delimiter).to_numpy()
    if ignore_columns != None:
        features = features[:, [x for x in range(features.shape[1]) if x not in ignore_columns]]
    if y_index == -1:
        y_index = features.shape[1] - 1
    X = features[:, [x for x in range(features.shape[1]) if x != y_index]]
    y = features[:, y_index]
    return X.astype('int64'), y

In [246]:
def readedges(file_path, adj_matrix=False):
    '''
    @ params
        file_path: a string indicate the path of the file
        adj_matrix: wether return the graph as adj matrix, default False
    '''
    G = nx.read_edgelist(file_path)
    if adj_matrix:
        return nx.adjacency_matrix(G).todense()
    return G

In [247]:
def encode_label(y):
    '''

    '''
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(y)
    labels = to_categorical(labels)
    return labels

In [278]:
#https://towardsdatascience.com/graph-convolutional-networks-on-node-classification-2b6bbec1d042
class cora_loader():
    def __init__(self, path_con, path_city):
        '''
            @ params：
                path_con: path for the file cora.content
                path_city: path for the file cora.cities
                ratio: a list represent [ratio of training, ratio of test]
        '''
        all_data = []
        all_edges = []
        with open(path_con) as f:
            all_data.extend(f.read().splitlines())
        with open(path_city) as f:
            all_edges.extend(f.read().splitlines())
        random.seed(4)
        random.shuffle(all_data)
        labels = []
        nodes = []
        X = []

        for i , data in enumerate(all_data):
            elements = data.split('\t')
            labels.append(elements[-1])
            X.append(elements[1:-1])
            nodes.append(elements[0])

        self.X = np.array(X,dtype=int)
        self.y = labels
        #parse the edge
        edge_list=[]
        for edge in all_edges:
            e = edge.split('\t')
            edge_list.append((e[0],e[1]))
        G = nx.Graph()
        G.add_nodes_from(nodes)
        G.add_edges_from(edge_list)

        #obtain the adjacency matrix (A)
        self.A = nx.adjacency_matrix(G)
    def get_train(self):
        return self.X, self.y, self.A

In [279]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, Callback
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from spektral.layers import GraphConv

In [293]:
class gnn_nlayer():
    def __init__ (self, channels = 16, dropout=0.1, l2_reg = 5e-4):
        '''
            channels: output channels of each layer
            dropout: dropout ratio of dropout layer
            l2_reg: if -1, then no reularization
        '''
        self.channels = channels
        self.drop_out = dropout
        self.l2_reg = l2_reg
        
    def fit(self, X, y, A, num_classes = 7, train_part = 0.5):
        '''
            X: features
            A: adj matrix
        '''
        self.A = GraphConv.preprocess(A).astype('f4')
        self.F = X.shape[1]
        self.N = X.shape[0]
        self.X = X
        self.y = y
        self.num_classes = num_classes
        
        #https://towardsdatascience.com/graph-convolutional-networks-on-node-classification-2b6bbec1d042
        train_idx = np.random.choice(len(self.y), int(np.ceil(train_part * len(self.y))), replace=False)

        #get the indices that do not go to traning data
        val_idx = [x for x in range(len(self.y)) if x not in train_idx]
        train_mask = np.zeros((self.N,),dtype=bool)
        train_mask[train_idx] = True
        self.train_mask = train_mask
        val_mask = np.zeros((self.N,),dtype=bool)
        val_mask[val_idx] = True
        self.val_mask = val_mask
        self.y = encode_label(self.y)
        
    def model(self, n):
        input_layer = Input(shape=(self.F, ))
        filter_in = Input((self.N, ), sparse=True)
        dropout = Dropout(self.drop_out)(input_layer)
        graph_conv_ = GraphConv(self.channels,
                             activation='relu',
                             use_bias=True,
                             kernel_regularizer=l2(self.l2_reg))([dropout, filter_in])
        for i in range(1, n):
            dropout = Dropout(self.drop_out)(graph_conv_)
            graph_conv_ = GraphConv(
                                     self.channels,
                                     kernel_regularizer=l2(self.l2_reg),
                                     use_bias=True,
                                     activation='relu')([dropout, filter_in]
                                    )

        dropout_ = Dropout(self.drop_out)(graph_conv_)
        graph_conv_ = GraphConv(self.num_classes,
                                 use_bias=True,
                                 activation='softmax')([dropout_, filter_in])
        model = Model(inputs=[input_layer, filter_in], outputs=graph_conv_)
        self.model = model
        return model
    
    def mcompile(self, optimizer='adam', loss='cr', learning_rate=1e-3):
        if optimizer == 'adam':
            optimizer = Adam(lr=learning_rate)
        if loss == 'cr':
            loss = 'categorical_crossentropy'
        self.model.compile(optimizer=optimizer,
                  loss=loss,
                  weighted_metrics=['acc'])
        display(self.model.summary())
        return self.model
    
    def train(self, epochs=50):
        validation_data = ([self.X, self.A], self.y, self.val_mask)
        hist = self.model.fit([self.X, self.A],
                  self.y,
                  sample_weight=self.train_mask,
                  epochs=epochs,
                  batch_size=self.N,
                  validation_data=validation_data,
                  shuffle=False)
        return hist

In [294]:
cora = cora_loader('data/cora.content', 'data/cora.cites')
model_2 = gnn_nlayer()
X, y, A = cora.get_train()
model_2.fit(X, y, A)

In [295]:
model_2.model(2)

In [296]:
model_2.mcompile()

Model: "functional_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout_56 (Dropout)            (None, 1433)         0           input_41[0][0]                   
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv_55 (GraphConv)       (None, 16)           22944       dropout_56[0][0]                 
                                                                 input_42[0][0]       

None

In [297]:
model_2.train(250)

Epoch 1/250
1/1 [==============================] - 1s 743ms/step - loss: 0.9981 - acc: 0.1263 - val_loss: 0.9942 - val_acc: 0.1736
Epoch 2/250
1/1 [==============================] - 0s 61ms/step - loss: 0.9944 - acc: 0.1595 - val_loss: 0.9914 - val_acc: 0.2341
Epoch 3/250
1/1 [==============================] - 0s 61ms/step - loss: 0.9915 - acc: 0.2334 - val_loss: 0.9888 - val_acc: 0.2925
Epoch 4/250
1/1 [==============================] - 0s 80ms/step - loss: 0.9887 - acc: 0.2629 - val_loss: 0.9861 - val_acc: 0.3250
Epoch 5/250
1/1 [==============================] - 0s 82ms/step - loss: 0.9849 - acc: 0.3560 - val_loss: 0.9834 - val_acc: 0.3442
Epoch 6/250
1/1 [==============================] - 0s 84ms/step - loss: 0.9832 - acc: 0.3501 - val_loss: 0.9806 - val_acc: 0.3575
Epoch 7/250
1/1 [==============================] - 0s 79ms/step - loss: 0.9796 - acc: 0.3796 - val_loss: 0.9777 - val_acc: 0.3641
Epoch 8/250
1/1 [==============================] - 0s 72ms/step - loss: 0.9758 - acc: 0.4

1/1 [==============================] - 0s 64ms/step - loss: 0.3281 - acc: 0.9003 - val_loss: 0.3625 - val_acc: 0.8700
Epoch 126/250
1/1 [==============================] - 0s 68ms/step - loss: 0.3258 - acc: 0.9040 - val_loss: 0.3602 - val_acc: 0.8693
Epoch 127/250
1/1 [==============================] - 0s 62ms/step - loss: 0.3227 - acc: 0.9032 - val_loss: 0.3578 - val_acc: 0.8700
Epoch 128/250
1/1 [==============================] - 0s 61ms/step - loss: 0.3173 - acc: 0.9077 - val_loss: 0.3556 - val_acc: 0.8700
Epoch 129/250
1/1 [==============================] - 0s 64ms/step - loss: 0.3126 - acc: 0.9136 - val_loss: 0.3535 - val_acc: 0.8700
Epoch 130/250
1/1 [==============================] - 0s 79ms/step - loss: 0.3121 - acc: 0.9106 - val_loss: 0.3513 - val_acc: 0.8708
Epoch 131/250
1/1 [==============================] - 0s 63ms/step - loss: 0.3113 - acc: 0.9055 - val_loss: 0.3493 - val_acc: 0.8715
Epoch 132/250
1/1 [==============================] - 0s 72ms/step - loss: 0.3166 - acc: 0.

1/1 [==============================] - 0s 65ms/step - loss: 0.1968 - acc: 0.9298 - val_loss: 0.2769 - val_acc: 0.8796
Epoch 250/250
1/1 [==============================] - 0s 68ms/step - loss: 0.1928 - acc: 0.9372 - val_loss: 0.2768 - val_acc: 0.8804


In [300]:
import json
import argparse
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='Running model')
    parser.add_argument('--model', type=str, default='graph', choices=['graph'],
                        help='model to use for training (default: nlayerGNN)')
    parser.add_argument('--layer_number', type=int, default=1,
                        help='input layer number for nlayerGNN')
    
    parser.add_argument('--dataset', type=str, default='cora', choices=['cora'],
                        help='data set type (default cora and only support cora now)')
    parser.add_argument('--cora_path', type=str, default='/data',
                        help='path for the cora dataset')
    parser.add_argument('--output_path', type=str, default='/config/model-output.json',
                        help='path for the output json file')
    
    parser.add_argument('--channels', type=int, default=16,
                        help='channels output of each layer (GNN) (default: 16)')
    parser.add_argument('--dropout', type=int, default=0.1,
                        help='dropout ratio in dropout layer (default: 0.1)')
    parser.add_argument('--l2_reg', type=int, default=5e-4,
                        help='l2 regularzaytion (default: 5e-4)')
    
    parser.add_argument('--epochs', type=int, default=50,
                        help='number of epochs to train (default: 50)')
    parser.add_argument('--lr', type=float, default=1e-3,
                        help='learning rate (default: 1e-3)')
    
    args = parser.parse_args()
    cora = cora_loader(args.cora_path + '/cora.content', args.cora_path + '/cora.cites')
    model = gnn_nlayer(channels = args.channels, dropout = args.dropout, l2_reg = args.l2_reg)
    X, y, A = cora.get_train()
    model.fit(X, y, A)
    model.mcompile(optimizer='adam', loss='cr', learning_rate=args.lr)
    hist = model.train(args.epochs)
    with open(args.output_path, 'w') as f:
        json.dump(hist.history, f)

if __name__ == '__main__':
    main()
    # Examples:
    # python run.py --model graph --dataset cora

usage: ipykernel_launcher.py [-h] [--model {graph}]
                             [--layer_number LAYER_NUMBER] [--dataset {cora}]
                             [--cora_path CORA_PATH]
                             [--output_path OUTPUT_PATH] [--channels CHANNELS]
                             [--dropout DROPOUT] [--l2_reg L2_REG]
                             [--epochs EPOCHS] [--lr LR]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/x5zhan/.local/share/jupyter/runtime/kernel-7dce758e-8ff2-4c92-8077-b00d08365723.json


SystemExit: 2

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
